# MS-GSP algorithm for generating sequential patterns
## Implemented by Firoz, Shaik and Charith Reddy, Gundala

Importing libraries

In [278]:
import re
from collections import defaultdict
import copy

In [279]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Function to read the data file

In [280]:
def extract_data_from_line(data_line):
    cleaned_line = data_line.strip().strip('<').strip('>')
    return [list(map(int, item.split(','))) for item in re.findall('{(.*?)}', cleaned_line)]

data_file_path = '/content/drive/MyDrive/large-data-2/data2.txt'
parsed_data = defaultdict(list)

with open(data_file_path) as file:
    for line_number, line in enumerate(file, start=1):
        parsed_data[line_number] = extract_data_from_line(line)

Function to read the parameter file

In [281]:
MS = {}
SDC = None

with open('/content/drive/MyDrive/large-data-2/para2-2.txt') as param_file:
    for line in param_file:
        if "MIS" in line:
            item_id = int(line[line.find("(") + 1:line.find(")")])
            min_support = float(line[line.find("=") + 1:].lstrip(" ").lstrip("\n"))
            MS[item_id] = min_support
        else:
            SDC = float(line[line.find("=") + 1:].lstrip())

## Get M - Line 1 sorts the items in ascending order according to their MIS values stored in MS

In [282]:
M = []

for key, value in sorted(MS.items(), key=lambda kv: kv[1]):
    M.append(key)

## Functions

makes the first pass over the sequence data using the function init-pass (line 2)

In [283]:
def init_pass(M, S, n, MS):
    count = defaultdict(int)
    count_L = defaultdict(int)
    L = []

    for i, sequence in S.items():
        for item in M:
            if item in sum(sequence, []):
                count[item] += 1
    for item in M:
        if float(count[item] / n) >= float(MS[item]):
            min_MIS, min_index = float(MS[item]), M.index(item)
            break
    for index in range(min_index, len(M)):
        if float(count[M[index]] / n) >= min_MIS:
            L.append(M[index])
            count_L[M[index]] = count[M[index]]
    return L, count_L

Frequent 1-sequences (F1) are obtained from L (line 3)

In [284]:
def generate_F1(L, MS, C, count, n):
    F1 = []
    for item in L:
        support_ratio = C[item] / n
        if float(support_ratio) >= MS[item]:
            item_string = "[" + str(item) + "]"
            count[item_string] = C[item]
            F1.append([item])
    return F1

level2-candidate-gen-SPM()

In [285]:
def level2_Candidate_gen_SPM(L, item_counts, MS, SDC, total_items):
    def remove_duplicates(candidates):
        return [c for c in candidates if (isinstance(c[0], int) and c[0] < c[1]) or isinstance(c[0], list)]

    if L:
        candidate_pairs = []
        for item in L:
            if item_counts[item] >= MS[item]:
                for next_item in L[L.index(item):]:
                    if (
                        item_counts[next_item] >= MS[item]
                        and abs((item_counts[next_item] / total_items) - (item_counts[item] / total_items))
                        <= SDC
                    ):
                        candidate_pairs.append([item, next_item])
                        candidate_pairs.append([[item], [next_item]])
                for next_item in L[: L.index(item)]:
                    if (
                        item_counts[next_item] >= MS[item]
                        and abs((item_counts[next_item] / total_items) - (item_counts[item] / total_items))
                        <= SDC
                    ):
                        candidate_pairs.append([item, next_item])
                        candidate_pairs.append([[item], [next_item]])

    cleaned_candidates = remove_duplicates(candidate_pairs)
    return cleaned_candidates

checking if a sequence is subsequence

In [286]:
def is_subsequence_check(candidate, seq):
    if isinstance(candidate[0], int):candidate = [candidate]
    index, flags = -1, [0] * len(seq)

    for element in candidate:
        if len(element) == len(set(element)):
            for i in range(index + 1, len(seq)):
              if set(element).issubset(set(seq[i])) and not flags[i]:
                  index, flags[i] = i, 1
                  break

    return sum(flags) == len(candidate)

Generate Fk by checking the minmis

In [287]:
def Fk_Generation(Candidates, MS, count, n):
    F = []
    for i in Candidates:
        temp = [item for sublist in i for item in sublist] if isinstance(i[0], list) else copy.deepcopy(i)
        min_MIS = float('inf')
        for item in temp:
            min_MIS = min(min_MIS, MS[item])
        if float(count[str(i)] / n) >= float(min_MIS):F.append(i)
    return F

MS_Candidate-gen-SPM function(line 7)

In [288]:
def MScandidate_gen_SPM(F, count_L, SDC, MS, n):
    def merge_lists(s):
        return sum(s, []) if type(s[0]) is list else copy.deepcopy(s)

    candidate_list = []


    def is_smallest(s, MS):
        def merge_and_copy(s2):
            return sum(s2, []) if type(s2[0]) is list else copy.deepcopy(s2)

        temp = merge_and_copy(s)
        if len(set(temp)) == 1:
            return False
        for item in temp:
            if MS[item] <= MS[temp[0]] and item != temp[0]:
                return False
        return True

    def length(s):
        def merge_and_copy(s):
            return sum(s, []) if type(s[0]) is list else copy.deepcopy(s)

        return len(merge_and_copy(s))


    def size(s):
        if type(s[0]) is list:
            return len(s)
        else:
            return 1



    def first_MIS(s1, s2):
        s1_temp = merge_lists(s1)
        s2_temp = merge_lists(s2)
        s1_temp_popped = copy.deepcopy(s1_temp)
        s1_temp_popped.pop(1)
        s2_temp_popped = s2_temp[:-1]

        if s1_temp_popped == s2_temp_popped and abs(float(count_L.get(s2_temp[-1]) / n) - float(
                count_L.get(s1_temp[1]) / n)) <= SDC and MS[s2_temp[-1]] >= MS[s1_temp[0]]:
            if isinstance(s2[-1], list) and length(s2[-1]) == 1:
                if isinstance(s1[-1], int):
                    candidate = []
                    candidate.append(copy.deepcopy(s1))
                    candidate.append(copy.deepcopy(s2[-1]))
                    candidate_list.append(candidate)
                else:
                    candidate = copy.deepcopy(s1) + [copy.deepcopy(s2[-1])]
                    candidate_list.append(candidate)
                if length(s1) == 2 and size(s1) == 2 and s2_temp[-1] > s1_temp[-1]:
                    candidate = copy.deepcopy(s1)
                    candidate[-1].extend(copy.deepcopy(s2[-1]))
                    candidate_list.append(candidate)
            elif (size(s1) == 1 and length(s1) == 2 and s2_temp[-1] > s1_temp[-1]) or length(s1) > 2:
                if type(s1[-1]) is int:
                    candidate = copy.deepcopy(s1)
                    candidate.append(s2[-1])
                    candidate_list.append(candidate)
                else:
                    last = copy.deepcopy(s2[-1])
                    if isinstance(last, list):
                        item = last[-1]
                    else:
                        item = last
                    candidate = copy.deepcopy(s1)
                    candidate[-1].append(item)
                    candidate_list.append(candidate)

    def last_MIS(s2, MS):
        def merge_and_copy(s2):
            return sum(s2, []) if type(s2[0]) is list else copy.deepcopy(s2)

        temp = merge_and_copy(s2)
        last_item = temp[-1]
        if len(set(temp)) == 1:
            return False
        for i in temp:
            if MS[i] <= MS[last_item] and i != last_item:
                return False
        return True

    def remove_last_MIS(s1, s2):
        s1_temp = merge_lists(s1)
        s2_temp = merge_lists(s2)
        s1_0 = s1_temp[0]
        s2_0 = s2_temp[0]
        s1_first_MIS = MS[s1_temp[0]]
        s2_last_MIS = MS[s2_temp[-1]]
        S1_first = s1_temp.pop(0)
        if len(s2_temp) == 2:
            S2_2nd_last = s2_temp.pop(0)
        else:
            S2_2nd_last = s2_temp.pop(-2)
        if s1_temp == s2_temp and abs(float(count_L[S1_first] / n) - float((count_L[S2_2nd_last] / n))) <= SDC and s2_last_MIS < s1_first_MIS:
            if type(s1[0]) is list and len(s1[0]) == 1:
                if type(s2[0]) is int:
                    candidate = []
                    candidate.append(copy.deepcopy(s1[0]))
                    candidate.append(copy.deepcopy(s2))
                    candidate_list.append(candidate)
                else:
                    candidate = [copy.deepcopy(s1[0])] + copy.deepcopy(s2)
                    candidate_list.append(candidate)
                if length(s2) == 2 and size(s2) == 2 and s1_0 < s2_0:
                    candidate = copy.deepcopy(s2)
                    candidate[0].insert(0, s1[0][0])
                    candidate_list.append(candidate)
            elif (length(s2) == 2 and size(s2) == 1 and s1_0 < s2_0) or length(s2) > 2:
                if type(s2[0]) is int:
                    candidate = copy.deepcopy(s2)
                    candidate.insert(0, s1[0])
                    candidate_list.append(candidate)
                else:
                    first = copy.deepcopy(s1[0])
                    if isinstance(first, list):
                        item = first[0]
                    else:
                        item = first
                    candidate = copy.deepcopy(s2)
                    candidate[0].insert(0, item)
                    candidate_list.append(candidate)

    def join_phase(s1, s2):
        s1_temp = merge_lists(s1)
        s2_temp = merge_lists(s2)
        S1_first = s1_temp.pop(0)
        S2_last = s2_temp.pop()
        if s1_temp == s2_temp and abs(float(count_L[S1_first] / n) - float((count_L[S2_last] / n))) <= SDC:
            last = copy.deepcopy(s2[-1])
            if type(last) is list and len(last) == 1:
                if type(s1[0]) is int:
                    candidate = []
                    candidate.append(copy.deepcopy(s1))
                    candidate.append(copy.deepcopy(s2[-1]))
                    candidate_list.append(candidate)
                else:
                    candidate = copy.deepcopy(s1) + [copy.deepcopy(s2[-1])]
                    candidate_list.append(candidate)
            else:
                if type(s1[0]) is int:
                    if isinstance(last, list):
                        item = last[-1]
                    else:
                        item = last
                    candidate = copy.deepcopy(s1)
                    candidate.append(item)
                    candidate_list.append(candidate)
                else:
                    if isinstance(last, list):
                        item = last[-1]
                    else:
                        item = last
                    candidate = copy.deepcopy(s1)
                    candidate[-1].append(item)
                    candidate_list.append(candidate)

    for s1 in F:
        for s2 in F:
            if is_smallest(s1, MS):
                first_MIS(s1, s2)
            elif last_MIS(s2, MS):
                remove_last_MIS(s1, s2)
            else:
                join_phase(s1, s2)

    twin = []
    for i in range(0, len(candidate_list) - 1):
        for j in range(i + 1, len(candidate_list) - 2):
            if candidate_list[i] == candidate_list[j]:
                twin.append(candidate_list[i])
    for i in twin:
        candidate_list.remove(i)
    for k in candidate_list:
        if not prune_phase(k, MS, F):
            candidate_list.remove(k)
    return candidate_list

def prune_phase(candidate, MS, F):
    def min_MIS(s, MS):
        def merge_and_copy(s2):
            return sum(s2, []) if type(s2[0]) is list else copy.deepcopy(s2)

        temp = merge_and_copy(s)
        min_MIS_val = float('inf')
        for item in temp:
            min_MIS_val = min(min_MIS_val, MS[item])
        return min_MIS_val

    min_MIS_value = min_MIS(candidate, MS)
    if isinstance(candidate[0], list):
        for i in range(len(candidate)):
            for item in range(len(candidate[i])):
                temp = copy.deepcopy(candidate)
                if len(candidate[i]) == 1:
                    temp.remove(candidate[i])
                    if len(temp) == 1:
                        temp = temp[0]
                else:
                    temp[i].remove(candidate[i][item])
                temp_mis = min_MIS(temp, MS)
                if temp_mis == min_MIS_value:
                    if temp not in F:
                        return False
    else:
        for i in range(len(candidate)):
            temp = copy.deepcopy(candidate)
            temp.remove(candidate[i])
            temp_mis = min_MIS(temp, MS)
            if temp_mis == min_MIS_value:
                if temp not in F:
                    return False
    return True

Creating output file

In [289]:
def print_output(k, F, count):
    with open("/content/drive/MyDrive/Result data/result-2-2.txt", "a") as file:
        file.write("\n**********************************************\n")
        file.write("\n{}-sequences:\n".format(k))
        for candidate in F:
            output_var = list(str(candidate))
            if isinstance(candidate[-1], list):output_var[0], output_var[-1] = '<', '>'
            else:output_var[0], output_var[-1] = '<{', '}>'
            for i in range(len(output_var)):
                if output_var[i] == '[':output_var[i] = '{'
                if output_var[i] == ']':output_var[i] = '}'
                if output_var[i] == ',':
                    if output_var[i-1] == '}':output_var[i], output_var[i+1]  = '', ''
            output_var = ''.join(output_var)
            file.write("\n{} -1 count: {}".format(output_var, count[str(candidate)]))
        file.write("\n\n The count of {}-sequences is: {}".format(k, len(F)))

## init-pass (line 2) - Getting L

In [290]:
n=len(parsed_data)
L, Count_L=init_pass(M, parsed_data, n, MS)

## Generating frequent 1-sequences from L (line 3)

In [291]:
F = defaultdict(list)
seq_number = 1
count = defaultdict(int)
F[seq_number] = generate_F1(L, MS, Count_L, count, n)
print_output(seq_number, F[seq_number], count)

## Generating F2, F3, ...(line 4 to end)

In [292]:
seq_number += 1
while F[seq_number - 1]:
    Candidates = []
    if seq_number == 2:
        Candidates = level2_Candidate_gen_SPM(L, Count_L, MS, SDC, n)
    else:
        Candidates = MScandidate_gen_SPM(F[seq_number - 1], Count_L, SDC, MS, n)

    for t, s in parsed_data.items():
        for candidate in Candidates:
            isSequence = is_subsequence_check(candidate, s)
            if isSequence:
                count[str(candidate)] += 1
    F[seq_number] = Fk_Generation(Candidates, MS, count, n)
    print_output(seq_number, F[seq_number], count)
    seq_number += 1